# K-Means

In [1]:
import numpy as np

In [2]:
# initialize data

data1 = np.array([[1,2],[1,3],[1,5],[2,3],[2,1],[6,7],[5,6],[5,9],[7,6],[8,6]])

print(data1)

[[1 2]
 [1 3]
 [1 5]
 [2 3]
 [2 1]
 [6 7]
 [5 6]
 [5 9]
 [7 6]
 [8 6]]


In [18]:
# kmeans(data, k)
# input: data = [[1,2]]
#        k = 1
# output: 
#
def kmeans(data, k):
    # Initialization
    centers = np.zeros((2,k))
    new_centers = np.zeros((2,k))

    sample_indices = np.random.choice(data.shape[0], k, replace=False)
    centers = data[sample_indices]
    print("Centers:\n", centers)

    # Learning
    for _ in range(20):
        # Compute Distances
        distances = np.ones((k,data.shape[0]))
        print("Distances:\n", distances)

        for x in range(0,k):
            for y in range(0,data.shape[0]):
                distance = np.linalg.norm(centers[x] - data[y])
                distances[x][y] = distance
        print("Distances:\n", distances)

        # Identify closest clusters
        centers = np.argmin(distances,axis=0)
        print("Centers:\n", centers)

        # Update centers
        for z in range(0,k):
            new_centers[z] = np.array(data[centers == z].mean(axis=0))
        print("New Centers:\n", new_centers)

        # Check for convergence

        centers = new_centers

    return



kmeans(data1, 2)

Centers:
 [[1 5]
 [6 7]]
Distances:
 [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
Distances:
 [[3.         2.         0.         2.23606798 4.12310563 5.38516481
  4.12310563 5.65685425 6.08276253 7.07106781]
 [7.07106781 6.40312424 5.38516481 5.65685425 7.21110255 0.
  1.41421356 2.23606798 1.41421356 2.23606798]]
Centers:
 [0 0 0 0 0 1 1 1 1 1]
New Centers:
 [[1.4 2.8]
 [6.2 6.8]]
Distances:
 [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
Distances:
 [[0.89442719 0.4472136  2.23606798 0.63245553 1.8973666  6.2289646
  4.81663783 7.16937933 6.4498062  7.33484833]
 [7.07672241 6.44049688 5.5027266  5.6639209  7.16100552 0.28284271
  1.44222051 2.50599282 1.13137085 1.96977156]]
Centers:
 [0 0 0 0 0 1 1 1 1 1]
New Centers:
 [[1.4 2.8]
 [6.2 6.8]]
Distances:
 [[1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1. 1. 1. 1. 1. 1.]]
Distances:
 [[0.89442719 0.4472136  2.23606798 0.63245553 1.8973666  6.2289646
  4.81663783 7.16937933 6.4498062  7.3348483